In [1]:
from bs4 import BeautifulSoup as bs
import lxml, requests
import pandas as pd


In [121]:

def scrap1(start_pg, end_pg):
    links = ["https://elcinema.com/index/work/country/eg?page=" + str(n) for n in range(start_pg, end_pg+1)]
    new_table = {'Index':[], 'Link':[], 'Title':[], 'Type':[], 'Year':[], 'Rating':[]}
    for link in links:
        pg = requests.get(link)
        soup = bs(pg.text)
        data = []
        table = soup.find('table', attrs={'class':'expand'})
        rows = table.find_all('tr')
        rows.pop(0)
        cols = []
        for row in rows:
            cols.append(row.find_all('td'))
        for col in cols:
            new_table['Index'].append(col[0].text.strip()[:-1]),
            new_table['Link'].append(col[1].find_all('a', href=True)[1]['href']),
            new_table['Title'].append(col[1].text.strip()),
            new_table['Type'].append(col[2].text.strip()),
            new_table['Year'].append(col[3].text.strip()),
            new_table['Rating'].append(col[4].text.strip().split()[0])
    df = pd.DataFrame.from_dict(new_table)
    dd = pd.read_json('elcinema.json')
    dd = dd.append(df, ignore_index=True)
    dd.to_json('elcinema.json')
    print(f'Done! {start_pg}-{end_pg}')

In [133]:
#scrap(501,502)

Done! 501-502


In [121]:

def scrap2(start_pg, end_pg):
    yr_wk = [ (w,y) for w in range(2011,2021) for y in range(1,53) ]
    for w in range(1,16):
        yr_wk.append((2021,w))
    links = [ f"https://elcinema.com/boxoffice/{y}/{w}" for y, w in yr_wk ]
    new_table = {'Index':[], 'Img':[], 'Link':[], 'Title':[], 'Year_Week':[], 'Week Revenue':[], 'Total Revenue':[],  'Rating':[]}
    for i, link in enumerate(links[start_pg : end_pg]):
        pg = requests.get(link)
        soup = bs(pg.text)
        table = soup.find('div', attrs={'class':'columns small-12 medium-7 large-8'})
        rows = table.find_all('div', attrs={'class':'row'})
        rows.pop(0)
        for row in rows:
            new_table['Index'].append(int(row.find('div', attrs={'class':'columns small-2 medium-1 large-1'}).text.strip()))
            new_table['Img'].append(row.find('img', attrs={'class':'lazy-loaded'}).get('data-src'))
            new_table['Link'].append(row.find_all('a')[4].get('href'))
            new_table['Title'].append(row.find_all('a')[4].text.strip())
            new_table['Week Revenue'].append(int("".join(row.find_all('li')[4].text.strip().split()[0].split(','))))
            new_table['Total Revenue'].append(int("".join(row.find_all('li')[6].text.strip().split()[0].split(','))))
            new_table['Rating'].append(float(row.find_all('a')[3].text.strip()))
            new_table['Year_Week'].append(yr_wk[i+start_pg])
    df = pd.DataFrame.from_dict(new_table)
    dd = pd.read_json('elcinema_rev.json')
    dd = dd.append(df, ignore_index=True)
    dd.to_json('elcinema_rev.json')
    print(f'Done!\npages: {start_pg}-{end_pg}.')


In [137]:
scrap2(500,536)

Done!
pages: 500-536.


In [138]:
dx = pd.read_json('elcinema_rev.json')
dx

,Index,Img,Link,Title,Year_Week,Week Revenue,Total Revenue,Rating
0,1,https://media.elcinema.com/uploads/_315x420_bd...,/work/1684488/,بلبل حيران,"[2011, 1]",852249,17543670,7.2
1,2,https://media.elcinema.com/uploads/_315x420_da...,/work/1411307/,ابن القنصل,"[2011, 1]",754077,11888857,8.2
2,3,https://media.elcinema.com/uploads/_315x420_19...,/work/1610187/,بون سواريه,"[2011, 1]",718571,4863597,3.7
3,4,https://media.elcinema.com/uploads/_315x420_42...,/work/1528600/,٦٧٨ (ستة - سبعة - ثمانية),"[2011, 1]",458906,4561164,5.8
4,5,https://media.elcinema.com/uploads/_315x420_db...,/work/1872179/,زهايمر,"[2011, 1]",154265,13875419,7.5
...,...,...,...,...,...,...,...,...
9269,21,https://media.elcinema.com/uploads/_315x420_4e...,/work/2060631/,The Host,"[2021, 15]",7251,73044,4.3
9270,22,https://media.elcinema.com/uploads/_315x420_59...,/work/2067998/,Don't Look Back,"[2021, 15]",5316,107418,4.3
9271,23,https://media.elcinema.com/uploads/_315x420_b6...,/work/2067226/,The 100 Candles Game,"[2021, 15]",4938,844052,5.0
9272,24,https://media.elcinema.com/uploads/_315x420_a6...,/work/2060630/,شاومينج,"[2021, 15]",2881,747716,6.5


In [136]:
yr_wk = [ (w,y) for w in range(2011,2021) for y in range(1,53) ]
for w in range(1,16):
    yr_wk.append((2021,w))
links = [ f"https://elcinema.com/boxoffice/{y}/{w}" for y, w in yr_wk ]
print(len(links))

535
